In [14]:
from PIL import Image
from PIL import ImageDraw
import hashlib

'''
def getRandomColor():
    c1 = random.randint(0,255)
    c2 = random.randint(0,255)
    c3 = random.randint(0,255)
    return (c1,c2,c3)

def addTransparency(img, factor = 0.1 ):
    img = img.convert('RGBA')
    img_blender = Image.new('RGBA', img.size, (0,0,0,0))
    img = Image.blend(img_blender, img, factor)
    return img
'''

MD5 = hashlib.md5('水印信息'.encode(encoding='UTF-8')).hexdigest()
#print(MD5)
 
    
# 获取一个Image对象，参数分别是RGB模式。宽850，高100，随机颜色
image = Image.new('RGB',(850,100),"white")
#image = addTransparency(img, factor = 0.1)
 
# 获取一个画笔对象，将图片对象传过去
draw = ImageDraw.Draw(image)
 
# 获取一个font字体对象参数是ttf的字体文件的目录，以及字体的大小
font=ImageFont.truetype("华文新魏.ttf",size=50)
 
# 在图片上写东西,参数是：定位，字符串，颜色，字体
draw.text((20,0), MD5, "black", font=font)
 
# 保存到硬盘，名为test.png格式为png的图片
image.save(open('waterwater.png','wb'),'png')

881af3fc9875476b7a23f4117d9716e7


In [21]:
import cv2
import numpy as np
import random
import os
from argparse import ArgumentParser
ALPHA = 5



def main():
    #parser = build_parser()
    #options = parser.parse_args()
    img = "telescope.jpg"
    wm = "waterwater.png"
    res = "encode3.png"
    alpha = 40
    if not os.path.isfile(img):
        parser.error("image %s does not exist." % img)
    if not os.path.isfile(wm):
        parser.error("watermark %s does not exist." % wm)
    encode(img, wm, res, alpha)


def encode(img_path, wm_path, res_path, alpha):
    img = cv2.imread(img_path)
    img_f = np.fft.fft2(img)
    height, width, channel = np.shape(img)
    watermark = cv2.imread(wm_path)
    wm_height, wm_width = watermark.shape[0], watermark.shape[1]
    x, y = list(range(int(height / 2))), list(range(width))
    random.seed(height + width)
    random.shuffle(x)
    random.shuffle(y)
    tmp = np.zeros(img.shape)
    for i in range(int(height / 2)):
        for j in range(width):
            if x[i] < wm_height and y[j] < wm_width:
                tmp[i][j] = watermark[x[i]][y[j]]
                tmp[height - 1 - i][width - 1 - j] = tmp[i][j]
    res_f = img_f + alpha * tmp
    res = np.fft.ifft2(res_f)
    res = np.real(res)
    cv2.imwrite(res_path, res, [int(cv2.IMWRITE_JPEG_QUALITY), 100])


if __name__ == '__main__':
    main()

In [25]:
import cv2
import numpy as np
import random
import os
from argparse import ArgumentParser
ALPHA = 5



def main():
    #parser = build_parser()
    #options = parser.parse_args()
    ori = "telescope.jpg"
    img = "Dim3.png"
    res = "Dimdecode3.png"
    alpha = 40
    if not os.path.isfile(ori):
        parser.error("original image %s does not exist." % ori)
    if not os.path.isfile(img):
        parser.error("image %s does not exist." % img)
    decode(ori, img, res, alpha)


def decode(ori_path, img_path, res_path, alpha):
    ori = cv2.imread(ori_path)
    img = cv2.imread(img_path)
    ori_f = np.fft.fft2(ori)
    img_f = np.fft.fft2(img)
    height, width = ori.shape[0], ori.shape[1]
    watermark = (ori_f - img_f) / alpha
    watermark = np.real(watermark)
    res = np.zeros(watermark.shape)
    random.seed(height + width)
    x = list(range(int(height / 2)))
    y = list(range(width))
    random.shuffle(x)
    random.shuffle(y)
    for i in range(int(height / 2)):
        for j in range(width):
            res[x[i]][y[j]] = watermark[i][j]
    cv2.imwrite(res_path, res, [int(cv2.IMWRITE_JPEG_QUALITY), 100])


if __name__ == '__main__':
    main()

In [5]:
import numpy as np
import random
import cv2
from matplotlib import pyplot as plt



def sp_noise(image,prob):

    '''

    添加椒盐噪声

    prob:噪声比例

    '''

    output = np.zeros(image.shape,np.uint8)
    thres = 1 - prob
    
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            rdn = random.random()
            if rdn < prob:
                output[i][j] = 0
            elif rdn > thres:
                output[i][j] = 255
            else:
                output[i][j] = image[i][j]
    return output



def gasuss_noise(image, mean=0, var=0.001):

    '''

        添加高斯噪声

        mean : 均值

        var : 方差

    '''

    image = np.array(image/255, dtype=float)
    noise = np.random.normal(mean, var ** 0.5, image.shape)
    out = image + noise

    if out.min() < 0:
        low_clip = -1.
    else:
        low_clip = 0.
    out = np.clip(out, low_clip, 1.0)
    out = np.uint8(out*255)

    #cv.imshow("gasuss", out)

    return out

# Read image

img = cv2.imread("encode.png")

# 添加椒盐噪声，噪声比例为 0.02

out1 = sp_noise(img, prob=0.02)

cv2.imwrite("Pepper.png",out1)

# 添加高斯噪声，均值为0，方差为0.001

out2 = gasuss_noise(img, mean=0, var=0.001)

cv2.imwrite("Gasuss.png",out2)


"\nplt.figure(1)\nplt.subplot(131)\nplt.axis('off')  # 关闭坐标轴\nplt.title('Original')\nplt.imshow(img)\nplt.subplot(132)\nplt.axis('off')\nplt.title('Add Salt and Pepper noise')\nplt.imshow(out1)\nplt.subplot(133)\nplt.axis('off')\nplt.title('Add Gaussian noise')\nplt.imshow(out2)\nplt.show()\n"

In [24]:
import cv2

import numpy as np

# motion filter

def motion_filter(img, K_size=3):

    H, W, C = img.shape

    # Kernel

    K = np.diag( [1] * K_size ).astype(np.float)

    K /= K_size

    # zero padding

    pad = K_size // 2

    out = np.zeros((H + pad * 2, W + pad * 2, C), dtype=np.float)

    out[pad: pad + H, pad: pad + W] = img.copy().astype(np.float)

    tmp = out.copy()

    # filtering

    for y in range(H):

        for x in range(W):

            for c in range(C):

                out[pad + y, pad + x, c] = np.sum(K * tmp[y: y + K_size, x: x + K_size, c])

    out = out[pad: pad + H, pad: pad + W].astype(np.uint8)

    return out

# Read image

img = cv2.imread("encode3.png")

# motion filtering

out = motion_filter(img, K_size=3)

# Save result

cv2.imwrite("Dim3.png", out)

#cv2.imshow("result", out)

#cv2.waitKey(0)

#cv2.destroyAllWindows()

True

In [3]:
import cv2
#
#图像翻转
#

image = cv2.imread("encode.png")

#图像水平翻转
flipped = cv2.flip(image,1)
cv2.imwrite("Flipped.png", flipped)

True

In [10]:
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import filedialog
import os 

global DIR_PATH
DIR_PATH = os.path.dirname(os.path.realpath("D:\Program Op\Jupyter\\"))

root = tk.Tk()
root.wm_geometry("4000x4000")

canvas = tk.Canvas(root, width=4000, height=4000)
canvas.grid(row=0, column=1, rowspan=20)
canvas_d = tk.Canvas(root, width=4000, height=4000)
canvas_d.grid(row=0, column=2, rowspan=20)

upload_button = tk.Button(root, text="Upload image", command=lambda: upload_img(canvas))
upload_button.grid(row=0, column=0, sticky=tk.NSEW)

dither_button = tk.Button(root, text="Dither", command=lambda: dither(1))
dither_button.grid(row=1, column=0, sticky=tk.NSEW)

save_button = tk.Button(root, text="Save image", command=lambda: save_img())
save_button.grid(row=2, column=0, sticky=tk.NSEW)



def closest_step(pixels, steps):
    return tuple([round(steps * pixel / 255) * int(255 / steps) for pixel in pixels])

def quant_error_calc(pixels, x, y, quant_error, factor):
    result = add(pixels[x, y], multiply(quant_error, factor))
    return int(result[0]), int(result[1]), int(result[2])

def dist_err(image, pixels, x, y, error):
    add_err(image, pixels, 7 / 16.0, x + 1, y, error)
    add_err(image, pixels, 3 / 16.0, x - 1, y + 1, error)
    add_err(image, pixels, 5 / 16.0, x, y + 1, error)
    add_err(image, pixels, 1 / 16.0, x + 1, y + 1, error)

def add_err(image, pixels, factor, x, y, error):
    if x <= 0 or x >= image.size[0] or y <= 0 or y >= image.size[1]:
        return
    try:
        r, g, b, a = pixels[x, y]
    except ValueError:
        r, g, b = pixels[x, y]
    pixels[x, y] = (int(r+error[0]*factor), int(g+error[1]*factor), int(b+error[2]*factor), 255)       

def dither(steps):
    try:
        image = Image.open(root.filename)
    except AttributeError:
        print('No image')
        return
    pixels = image.load()
    for x in range(image.size[0]):
        for y in range(image.size[1]):
            old_pixel = pixels[x, y]
            new_pixel = closest_step(old_pixel, steps)

            pixels[x, y] = new_pixel

            error = tuple([old_pixel[0] - new_pixel[0], old_pixel[1] - new_pixel[1], old_pixel[2] - new_pixel[2]])

            dist_err(image, pixels, x, y, error)

    draw(image, 0)

def draw(image, i):
    global tk_img, imagesprite, imagesprite_d, tk_img_d, dithered
    if i:
        tk_img = ImageTk.PhotoImage(image) 
        imagesprite = canvas.create_image(350, 350, image=tk_img)
    else:
        tk_img_d = ImageTk.PhotoImage(image)
        imagesprite_d = canvas_d.create_image(350, 350, image=tk_img_d)
        dithered = image

def upload_img(canvas):
    root.filename = filedialog.askopenfilename(initialdir = DIR_PATH,title = "Select file",filetypes = (("JPG","*.jpg"),("PNG","*.png")))
    draw(Image.open(root.filename), 1)

def save_img():
    try:
        root.filename1 = filedialog.asksaveasfilename(initialdir = DIR_PATH,title = "Select file",filetypes = (("Png","*.png"),("All types","*.all")))
        dithered.save(root.filename1 + '.png')
    except (NameError, KeyError) as e:
        print('Error saving')

root.mainloop()

In [22]:
import cv2


def function(image1,image2):
    h1,w1,c1 = image1.shape
    h2,w2,c2 = image2.shape
    
    if c1 != c2:
        print("channels NOT match, cannot merge")
        return 
    else:
        image3 = np.hstack([image2,image1])
        
    return image3


img = cv2.imread("encode3.png")
print(img.shape)
cropped1 = img[0:1080, 0:853]  # 裁剪坐标为[y0:y1, x0:x1]
cropped2 = img[0:1080, 853:3412]

cropped = function(cropped1, cropped2)
cv2.imwrite("cutting3.png", cropped)

(1080, 3412, 3)


True

In [16]:
import cv2
import numpy as np
img=cv2.imread('encode.png',1)
img_info=img.shape
image_height=img_info[0]
image_weight=img_info[1]

dst=np.zeros(img.shape,np.uint8)
for i in range(image_height-200):
    for j in range(image_weight-100):
        dst[i+200,j+100]=img[i,j]
cv2.imwrite('Dst.png',dst)

True

Missing arg...


SystemExit: 1